In [225]:
import pandas as pd
import numpy as np
import scipy.stats as sts

In [121]:
data = pd.read_csv("illiteracy.txt", sep="\t")
data.head()

,Country,Illit,Births
0,Albania,20.5,1.78
1,Algeria,39.1,2.44
2,Bahrain,15.0,2.34
3,Belize,5.9,2.97
4,Benin,73.5,5.60


In [122]:
data[["Illit", "Births"]].corr()

,Illit,Births
Illit,1.000000,0.768663
Births,0.768663,1.000000


In [123]:
round(0.768663,4)

0.7687

In [124]:
data[["Illit", "Births"]].corr(method="spearman")
# ?pd.DataFrame.corr()

,Illit,Births
Illit,1.000000,0.752962
Births,0.752962,1.000000


In [125]:
round(0.752962,4)

0.753

In [126]:
water = pd.read_csv("water2.txt", sep="\t")
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [127]:
water[["hardness", "mortality"]].corr()

,hardness,mortality
hardness,1.000000,-0.654849
mortality,-0.654849,1.000000


In [18]:
round(-0.654849,4)

-0.6548

In [19]:
water[["mortality", "hardness"]].corr(method="spearman")

,mortality,hardness
mortality,1.000000,-0.631665
hardness,-0.631665,1.000000


In [20]:
round(-0.631665,4)

-0.6317

In [128]:
w_north = water[water["location"]=="North"]
w_south = water[water["location"]=="South"]

In [129]:
w_north.corr()

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


In [130]:
w_south.corr()

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


In [183]:
round(-0.368598,4)

-0.3686

### Matthews Correlation

In [177]:
# bar task 

# once per month assume 1
once_man = 203 
once_woman = 239

# rare than month assume 0
man = 718 
woman = 515


mw = pd.DataFrame({"man": [man, once_man], "woman": [woman, once_woman ]})
mw

,man,woman
0,718,515
1,203,239


In [178]:
a,b,c,d = mw.man[0], mw.woman[0], mw.man[1], mw.woman[1]
MCC = ((a * d) - (b * c)) / ((a + b) * (a + c) * (b + d)* (c + d))**0.5
MCC

0.10900237458678963

In [170]:
round(MCC, 3)

0.109

In [167]:
sts.chi2_contingency(mw)
# ?sts.chi2_contingency

(19.40753078854304,
 1.0558987006638725e-05,
 1L,
 array([[243.03402985, 198.96597015],
        [677.96597015, 555.03402985]]))

In [216]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = sts.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

fem = np.append(np.ones(203), np.zeros(718))
mal = np.append(np.ones(239), np.zeros(515))
# bar_confint = proportions_confint_diff_ind(sample)

TypeError: data type not understood

In [191]:
fem_mal_ = proportions_confint_diff_ind(fem, mal)
round(fem_mal_[0],4)

-0.1392

In [150]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - sts.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return sts.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - sts.norm.cdf(z_stat)

In [151]:
man_sample = np.array([0] * man + [1] * once_man)
woman_sample = np.array([0] * woman + [1] * once_woman)

Z = proportions_diff_z_stat_ind(man_sample, woman_sample)

In [152]:
proportions_diff_z_test(Z)

8.153453089576601e-06

In [101]:
# Task General Social Survey 
cols = ["Не доволен","Более или менее", "Доволен"]
vals = [[197,111,33], [382,685,331],[110,342,333]]
index=["Не очень счастлив","Достаточно счастлив","Очень счастлив"]

survey = pd.DataFrame(vals, columns=cols, index=index)
survey

,Не доволен,Более или менее,Доволен
Не очень счастлив,197,111,33
Достаточно счастлив,382,685,331
Очень счастлив,110,342,333


In [105]:
survey_chi = sts.chi2_contingency(survey)
# ?sts.chi2_contingency(survey)
survey_chi

(293.68311039689746,
 2.4964299580093467e-62,
 4L,
 array([[ 93.08597464, 153.74722662,  94.16679873],
        [381.6251981 , 630.318542  , 386.0562599 ],
        [214.28882726, 353.93423138, 216.77694136]]))

In [106]:
round(survey_chi[0],4)

293.6831

In [214]:
def cramer_test(chi_sq, conf_mtx):
    n = sum(conf_mtx.sum())
    return np.sqrt(chi_sq/(n * (min(conf_mtx.shape) - 1)))

phi = cramer_test(survey_chi[0],survey)
round(phi,4)

0.2412

### Multiple hypothesis test

In [224]:
auc_score = pd.read_csv("AUCs.txt", sep="\t")
auc_score.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [265]:
clsf = auc_score.drop("Unnamed: 0", axis=1)

aucs_comp = list()

for i in clsf.columns:
    for j in clsf.columns:
        if i == j:
            continue
        stats, pvalue = sts.wilcoxon(clsf[i], clsf[j])
        aucs_comp.append([i, j, pvalue])
aucs_df = pd.DataFrame.from_records(aucs_comp)
aucs_df.drop_duplicates(subset=2,inplace=True)
aucs_df

,0,1,2
0,C4.5,C4.5+m,0.010757
1,C4.5,C4.5+cf,0.861262
2,C4.5,C4.5+m+cf,0.015906
4,C4.5+m,C4.5+cf,0.046333
5,C4.5+m,C4.5+m+cf,0.327826
8,C4.5+cf,C4.5+m+cf,0.022909


In [266]:
aucs_df[aucs_df[2]== min(aucs_df[2])]

,0,1,2
0,C4.5,C4.5+m,0.010757


In [267]:
aucs_df[aucs_df[2]<= 0.05]

,0,1,2
0,C4.5,C4.5+m,0.010757
2,C4.5,C4.5+m+cf,0.015906
4,C4.5+m,C4.5+cf,0.046333
8,C4.5+cf,C4.5+m+cf,0.022909


In [255]:
?sts.wilcoxon(clsf[i], clsf[j])

In [281]:
from statsmodels.sandbox.stats.multicomp import multipletests 


reject, p_corrected, a1, a2 = multipletests(aucs_df[2], 
                                            alpha = 0.05, 
                                            method = 'holm') 
aucs_df['p_corrected'] = p_corrected
aucs_df['reject'] = reject

In [282]:
aucs_df[aucs_df.reject == True]

,0,1,2,p_corrected,reject


In [277]:
?multipletests(aucs_df[2])                       

In [283]:
reject, p_corrected, a1, a2 = multipletests(aucs_df[2], 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
aucs_df['p_corrected'] = p_corrected
aucs_df['reject'] = reject
aucs_df[aucs_df.reject == True]

,0,1,2,p_corrected,reject
0,C4.5,C4.5+m,0.010757,0.045818,True
2,C4.5,C4.5+m+cf,0.015906,0.045818,True
8,C4.5+cf,C4.5+m+cf,0.022909,0.045818,True
